### Final Test

Our examples of finding the lowest-rated movies were polluted with movies rated by one or two people.

YOUR GOAL: Modify both Demo Spark 1 and Demo Spark 2 scripts to only consider movies with at least ten ratings. 

HINTS:
* RDD's have a filter function you can use.

 * It takes a function as a parameter, which accepts the entire key/value pair. 
 * So if you are calling filter() on an RDD that contains (movie_id, (sumOfRatings, totalRatings)), a lambda function that takes in "x" would refer to totalRatings as x[1][1]. x[1] gives us the "value" of (sumOfRatings, totalRatings) and x[1][1] pulls out totalRatings. 
 * This function should be an expression that returns True if the row should be kept, or False if it should be discarded. 

* DataFrames also have a filter() function. 
 * It is easier, you just pass in a string expression for what you want to filter on.
 * For example: df.filter("count > 10") would only pass through rows where the "count" column is greater than 10.

If you havent uploaded u.data and u.item, you can follow the below steps:

In Databricks, you can easily create a table from a data file (e.g. csv). 

Go to the Data section, click on the default database and then click on the + (plus) sign at the top to create a new table.

![table1](https://s3-us-west-1.amazonaws.com/julienheck/hadoop/9_final_test/databricks+-+create+table.png)

For the Data source, select "Upload file" and click on "Drop file or click here to upload". Browse to the file you want to upload.

![table2](https://s3-us-west-1.amazonaws.com/julienheck/hadoop/9_final_test/databricks+-+create+table+2.png)

Once the file has been uploaded, click on "Create Table with UI".

![table3](https://s3-us-west-1.amazonaws.com/julienheck/hadoop/9_final_test/databricks+-+create+table+3.png)

Select a running Cluster and click on "Preview Table".

Finally, you should be able to specify the table attributes, such as the table and the columns name and the data type of each column.

![table4](https://s3-us-west-1.amazonaws.com/julienheck/hadoop/9_final_test/databricks+-+create+table+4.png)



For `u.item` file, name the first 3 colums as follow: `movie_id`, `title`, `release_date`. Name the table `u_item`.
Data type is `string` by default, update it to `int` for `movie_id`.

For `u.data` file, name the colums as follow: `user_id`, `movie_id`, `rating`, `timestamp`. Name the table `u_data`.
Update data type to `int` for `user_id`, `movie_id` and `rating`.

Click on "Create Table" once you are done. You can now access the table in Databricks using Spark SQL commands.

In [3]:
#List the path of the u.item and u.data

display(dbutils.fs.ls("/FileStore/tables/"))

In [4]:
### Code for your script with RDD ###

rawItem = sc.textFile("dbfs:/FileStore/tables/u.item")
movieList = rawItem.map(lambda line: line.split("|")).map(lambda x: (int(x[0]), x[1]))

rawData = sc.textFile("dbfs:/FileStore/tables/u.data")
movieRatings = rawData.map(lambda line: line.split("\t")).map(lambda x: (int(x[1]), (int(x[2]), 1.0)))

### Write your steps below ###

ratingTotalsAndCount = movieRatings.reduceByKey(lambda movie1, movie2 : (movie1[0]+movie2[0], movie1[1]+movie2[1]))
averageRatings = ratingTotalsAndCount.filter(lambda x:x[1][1] > 10).mapValues(lambda totalAndCount: totalAndCount[0] / totalAndCount[1])
sortedMovies = averageRatings.sortBy(lambda x: x[1])
joinRDD = averageRatings.join(movieList)
listSorted = joinRDD.map(lambda x: (x[1]))
results = listSorted.sortBy(lambda x: x[0])
results.collect()


In [5]:
### Code for your script with DataFrames ###

itemDF = sqlContext.sql("SELECT movie_id, title FROM u_item")
dataDF = sqlContext.read.format("csv").options(inferschema='true', delimiter="\t").load("dbfs:/FileStore/tables/u.data")

### Write your steps below ###

dataDF = sqlContext.sql("SELECT user_id, movie_id, rating FROM u_data")
movieDF = dataDF.select("movie_id", "rating").cache()
averageRatings = movieDF.groupBy("movie_id").avg("rating")
counts = movieDF.groupBy("movie_id").count() 
averageAndCounts = counts.join(averageRatings, "movie_id").join(itemDF, "movie_id")
result = averageAndCounts.filter('count > 10')
topTen = result.orderBy("avg(rating)").select("title", "avg(rating)", "count").show(10)



Once you are done, download the notebook (click on File -> Export -> IPython notebook) and submit it through Canvas.